[[이유한님] 캐글 코리아 캐글 스터디 커널 커리큘럼](https://kaggle-kr.tistory.com/32)  
[2nd level. Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction)  
[XGBoost CV (LB .284)](https://www.kaggle.com/code/aharless/xgboost-cv-lb-284)

Based on [olivier's script](https://www.kaggle.com/code/ogrellier/xgb-classifier-upsampling-lb-0-283)

In [1]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50
# Note: I set EARLY_STOPPING_ROUNDS high so that (when OPTIMIZE_ROUNDS is set)
#       I will get lots of information to make my own judgment. You should probably
#       reduce EARLY_STOPPING_ROUNDS if you want to do actual early stopping.

I recommend initially setting `MAX_ROUNDS` fairly high and using `OPTIMIZE_ROUNDS` to get an idea of the appropriate number of rounds (which, in my judgment, should be close to the maximum value of `best_ntree_limit` among all folds, maybe even a bit higher if your model is adequately regularized...or alternatively, you could set `verbose=True` and look at the details to try to find a number of rounds that works well for all folds). Then I  ould turn off `OPTIMIZE_ROUNDS` and set `MAX_ROUNDS` to the appropraite number of total rounds.  
  
The problem with "early stopping" by choosing the best round for each fold is that it overfits to the validation data. It's therefore liable not to produce the optimal model for predicting test data, and if it's used to produce validation data for stacking/ensembling with other models, it would cause this one to have too much weight in the ensemble. Another possibility (and the default for XGBoost, it seems) is to use the round where the early stop actually happens (with the lag that verifies lack of improvement) rather than the best round. That solves the overfitting problem (provided the lag is long enough), but so far it doesn't seem to have helped. (I got a worse validation score with 20-round early stopping per fold than with a constant number of rounds for all folds, so the early stopping actually seemed to underfit.)  
  
__DeepL 번역__
처음에는 `MAX_ROUNDS`를 상당히 높게 설정하고 `OPTIMIZE_ROUNDS`를 사용하여 적절한 라운드 수를 파악하는 것이 좋습니다(제 판단으로는 모든 폴드 중 `best_ntree_limit`의 최대 값에 가까워야 하며, 모델이 적절하게 정규화된 경우 약간 더 높을 수도 있습니다... 또는 `verbose=True`를 설정하고 세부 사항을 살펴 모든 폴드에 잘 맞는 라운드 수를 찾을 수 있습니다). 그런 다음 `OPTIMIZE_ROUNDS`를 끄고 `MAX_ROUNDS`를 적절한 총 라운드 수로 설정합니다.  
  
각 폴드에 가장 적합한 라운드를 선택하여 "early stopping"하는 경우의 문제점은 유효성 검사 데이터에 과도하게 적합하다는 것입니다. 따라서 테스트 데이터 예측을 위한 최적의 모델을 생성하지 못할 수 있으며, 다른 모델과 stacking/ensembling을 위한 검증 데이터를 생성하는 데 사용하면 이 모델이 앙상블에서 너무 많은 가중치를 갖게 될 수 있습니다. 또 다른 가능성(그리고 XGBoost의 기본값인 것 같습니다)은 최상의 라운드가 아니라 실제로 조기 정지가 발생하는 라운드(개선 부족을 확인하는 지연이 있는 라운드)를 사용하는 것입니다. 이렇게 하면 과적합 문제를 해결할 수 있지만(지연이 충분히 길다면) 지금까지는 도움이 되지 않는 것 같습니다. (모든 폴드에 대해 일정한 라운드 수를 적용하는 것보다 폴드당 20라운드 조기 정지를 적용했을 때 더 나쁜 검증 점수를 받았기 때문에 실제로 조기 정지가 과적합한 것으로 보입니다.)

In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit   # jit( just-in-time) 컴파일러로써 데코레이트한 함수가 파이썬을 bytecode로 읽어서 빠르다 한다.
import time
import gc

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Compute gini

# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation
@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true) # np.asarray: 기본적으로 copy되지 않고 같은 메모리를 사용.
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i*delta
        delta += 1-y_i
    gini = 1-2*gini/(ntrue*(n-ntrue))
    return gini

https://www.kaggle.com/code/aharless/xgboost-cv-lb-284